In [3]:
from google.colab import drive 
drive.mount('gdrive')

Mounted at gdrive


In [4]:
import pandas as pd
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import layers, models, backend as K, callbacks
from keras.layers import LSTM
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import layers, models, backend as K, callbacks
from keras.layers import LSTM
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn import preprocessing
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from keras import regularizers


In [ ]:
stock_market_df = pd.read_csv('/content/gdrive/MyDrive/Stock_market_label_data.csv')
stock_market_df.head(10)

In [6]:
stock_market_df.drop(columns=['Company_Name', 'Unnamed: 0', 'Date', 'Name'], inplace=True)
stock_market_df['Match'].replace('NAN', -10, inplace=True)

In [7]:
len(stock_market_df.columns)

83

In [8]:
y = stock_market_df.iloc[:, -1].astype(int).values
stock_market_df_1 = stock_market_df.drop(columns=['Match'])
X=stock_market_df_1.values

In [9]:
X = np.array(X)
y = np.array(y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=False
)

In [ ]:
X_train

In [12]:
y_train

array([ 1,  1,  1, ..., -1, -1, -1])

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, shuffle=False
)

In [301]:
X_valid.shape

(1429, 82)

In [14]:
win_length = 60
batch_size = 32
num_features = 82
train_generator = TimeseriesGenerator(np.array(X_train), np.array(y_train),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

test_generator = TimeseriesGenerator(np.array(X_test), np.array(y_test),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

valid_generator = TimeseriesGenerator(np.array(X_valid), np.array(y_valid),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

In [29]:
model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(win_length, num_features), return_sequences=True))

In [15]:
model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(win_length, num_features), return_sequences=True))
model.add(layers.LeakyReLU(alpha=0.5))
model.add(layers.Dropout(0.3))
model.add(layers.LSTM(64, return_sequences=False))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
optimizer = keras.optimizers.Adam()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='',
                                                 patience=20,
                                                 mode="min")
model.compile(optimizer=optimizer, loss="categorical_crossentropy", 
                   metrics=["acc"])

model.fit_generator(train_generator,
                   epochs=10,
                   validation_data=valid_generator,
                   shuffle=False)


# ***New Data***

In [ ]:
stock_market_df = pd.read_csv('/content/gdrive/MyDrive/Stock_market_label_data.csv')
stock_market_df.head(10)

In [ ]:
stock_market_df.shape

In [37]:
stock_market_df.drop(columns=['Company_Name', 'Unnamed: 0', 'Date', 'Name'], inplace=True)
stock_market_df['Match'].replace('NAN', -10.0, inplace=True)
stock_market_df = stock_market_df[: 15862]

In [38]:
match_df = stock_market_df.iloc[:, -1:]

In [ ]:
match_df.info

In [40]:
df1 = match_df['Match']
df1 = pd.concat([df1, df1.shift(1)], axis=1)
df1.dropna(inplace=True)

In [41]:
df1 = df1.astype(float)

In [42]:
values = df1.values

In [ ]:
values

In [45]:
encoder = preprocessing.LabelEncoder()
test_labels = to_categorical(encoder.fit_transform(values[:,-1]),num_classes=3)
train_X,test_X,train_y,test_y= train_test_split(values[:,0], test_labels,test_size = 0.30,random_state = 42)

In [46]:
train_X.shape

(11102,)

In [47]:
test_X.shape

(4759,)

In [48]:
train_y.shape

(11102, 3)

In [49]:
test_y.shape

(4759, 3)

In [50]:
train_X = train_X.reshape(train_X.shape[0],1,1)
test_X = test_X.reshape(test_X.shape[0],1,1)

In [51]:
train_y.shape

(11102, 3)

In [52]:
test_X.shape

(4759, 1, 1)

In [53]:
train_X.shape

(11102, 1, 1)

In [376]:
n_batch = 1
n_epoch = 10
n_neurons = 100

In [379]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(n_neurons, batch_input_shape=(n_batch, train_X.shape[1],train_X.shape[2]), stateful=True),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(100, activation = 'relu',kernel_regularizer=regularizers.l2(0.0001)),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(train_X,train_y,validation_data=(test_X, test_y),epochs=n_epoch, batch_size=n_batch, verbose=1,shuffle= False)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (1, 100)                  40800     
_________________________________________________________________
dense (Dense)                (1, 64)                   6464      
_________________________________________________________________
dense_1 (Dense)              (1, 100)                  6500      
_________________________________________________________________
dense_2 (Dense)              (1, 3)                    303       
Total params: 54,067
Trainable params: 54,067
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11102/11102 [==============================] - 37s 3ms/step - loss: 0.6813 - acc: 0.6072 - val_loss: 0.6676 - val_acc: 0.6260
Epoch 2/10
11102/11102 [==============================] - 35s 3ms/step - loss: 0.6746 - acc: 0.6121 - val_loss: 0